In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
torch.manual_seed(1)
import os,csv,math, sys,joblib,copy,pickle
import matplotlib.pyplot as plt
import numpy as np
import sklearn.model_selection, sklearn.preprocessing

In [2]:
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

In [3]:
dataset = pd.read_csv('Iris_Dataset.csv')

In [4]:
dataset.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [4]:
dataset = pd.get_dummies(dataset, columns=['Species']) # One Hot Encoding
values = list(dataset.columns.values)

In [6]:
dataset.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


values

In [5]:
y = dataset[values[-3:]]
y = np.array(y, dtype='float32')
#y

In [6]:
X = dataset[values[:-3]]
print X.head()
X = np.array(X, dtype='float32')

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2


In [7]:
mean_vector=np.mean(X,axis=0)
#means.shape#4
cov=np.cov(X.T)
cov.shape

(4, 4)

X = dataset[values[1:-3]]
print X.head()
X = np.array(X, dtype='float32')

?BUT WHY IS THIS NOT INCLUDING 1ST COLUMN THAT IS sepalLenghtcm?

In [8]:
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]

scaler = joblib.load('scaler.pkl') 
X_values = scaler.transform(X_values)
y_values = y[indices]

for class_i in range(0, 3):
    print(np.array([feature_vector[1] for feature_vector in enumerate(X_values) if np.where(y_values[feature_vector[0]] == 1.0)[0] == class_i]).shape)

?Should we really divide the data according to class?

means = []
cov= []
for class_i in range(3):
    temp = np.array([feature_vector[1] for feature_vector in enumerate(X_values) if np.where(y_values[feature_vector[0]] == 1.0)[0] == class_i])
    #temp has data for a given class
    means.append(temp.mean(axis=0))
    cov.append(np.cov(temp))
means = np.array(means)

class_name = {0:"Iris-setosa",1:"Iris-versicolor",2:"Iris-virginica"}
feature_name = {0:"SepalLengthCm",1:"SepalWidthCm",2:"PetalLengthCm",3:"PetalWidthCm",4:"Species"}

In [9]:
class neural_network(nn.Module):
    def __init__(self, num_input, num_hidden,num_output):
        super(neural_network, self).__init__()
        self.hidden = nn.Linear(num_input,num_hidden).cuda()
        self.out = nn.Linear(num_hidden,num_output).cuda()

    def forward(self, x):
        x = F.relu(self.hidden(x))
        return self.out(x)

In [10]:
model = torch.load("iris_model_2").cpu()

In [27]:
print model

neural_network(
  (hidden): Linear(in_features=3, out_features=2, bias=True)
  (hidden_2): Linear(in_features=2, out_features=2, bias=True)
  (out): Linear(in_features=2, out_features=3, bias=True)
)


In [11]:
n_classes=3
num_c=4#no. of features
num_alpha=100

for output_index in range(0,n_classes):
    for t in range(0,num_c):
        expected_value = 0.0
        counter = 1
        expectation_do_x = []
        baseline_expectation_do_x = []
        inp=copy.deepcopy(mean_vector)
        
        for x in np.linspace(0, 1, num_alpha):
            inp[t] = x
            input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)
            
            output=F.softmax(model(input_torchvar), dim=-1) 
            o1=output.data.cpu()
            val=o1.numpy()[output_index]                                              
            grad_mask_gradient = torch.zeros(n_classes)
            grad_mask_gradient[output_index] = 1.0
            #calculating the hessian
            first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

            for dimension in range(num_c):#Hessian*Covariance
                if dimension == t:
                        continue
                temp_cov = copy.deepcopy(cov)
                temp_cov[dimension][t] = 0.0
                grad_mask_hessian = torch.zeros(num_c)#SHOULD IT BE num_classes
                grad_mask_hessian[dimension] = 1.0

                #calculating the hessian
                hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)

                val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])  #Has interventional expectations of x on y for an alpha
            expectation_do_x.append(val)

            expected_value += (1.0/counter)*(val - expected_value)
            counter += 1
        baseline_expectation_do_x.append(expected_value)
        with open('ACE/expectation_do_x_for_cause_'+str(t) + "_and_effect_" + str(output_index), 'wb') as fp:
            pickle.dump(expectation_do_x, fp)
        with open('ACE/baseline_expectation_do_x_for_cause_'+str(t) + "_and_effect_" + str(output_index), 'wb') as fp:
            pickle.dump(baseline_expectation_do_x, fp)

In [35]:
import sys

def causal_strength(model, feature_index, output_index, mean_data, cov_data,start,end):
    input_data = mean_data
    #print input_data.shape
    model.zero_grad()

    input_torchvar = autograd.Variable(torch.FloatTensor(input_data), requires_grad=True)

    output_2 = model(input_torchvar)	

    grad_mask = torch.zeros(output_2.size()).type(torch.FloatTensor)

    grad_mask[output_index] = 1.0

    first_grads = torch.autograd.grad(output_2, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=False)

    feature_index = (feature_index)
    #print feature_index

    causal_effect =  float(first_grads[0][feature_index].data)

    #print causal_effect
    #calculating the hessian (selective-terms-only)
    first_grad_shape = first_grads[0].data.size()

    grad_mask = torch.zeros(first_grad_shape)
    grad_mask[feature_index] = 1.0

    second_grads = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=False) 
    second_grads_array = np.array(second_grads[0].data)	

    causal_effect -=  float(second_grads[0][feature_index].data)*mean_data[feature_index]
    #print causal_effect

    causal_effect_x = float(second_grads[0].data[feature_index])
    #print "x", causal_effect_x

    class_name = {0:"Iris-setosa", 1:"Iris-versicolor", 2:"Iris-virginica"}

    grad_mask = torch.zeros(first_grad_shape)
    grad_mask[feature_index] = 1.0

    third_grads = torch.autograd.grad(second_grads, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=False)  


    third_grads_array = np.array(third_grads[0].data)

    causal_effect += 0.5*float(third_grads_array[feature_index])*mean_data[feature_index]*mean_data[feature_index] 
    causal_effect_x -= third_grads_array[feature_index]*mean_data[feature_index]
    causal_effect_x_2 = 0.5*float(third_grads_array[feature_index])


    grad_mask = torch.zeros(first_grad_shape)
    grad_mask[feature_index] = 1.0

    fourth_grads = torch.autograd.grad(third_grads, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=False)

    print(fourth_grads)
    
    fourth_grads_array = np.array(fourth_grads[0].data)

    #cov_temp_array = cov_data[i*first_grad_shape[1] + j].reshape(third_grads_array.shape)			 
    #cov_temp_array[feature_index] = 0.0

    causal_effect -= (1.0/6.0)*float(fourth_grads_array[feature_index])*mean_data[feature_index]*mean_data[feature_index]*mean_data[feature_index] 
    causal_effect_x += (0.5)*float(fourth_grads_array[feature_index])*mean_data[feature_index]*mean_data[feature_index]
    causal_effect_x_2 -= 0.5*float(fourth_grads_array[feature_index])*mean_data[feature_index]
    causal_effect_x_3 = (1.0/6.0)*float(fourth_grads_array[feature_index])



    grad_mask = torch.zeros(first_grad_shape)
    grad_mask[feature_index] = 1.0

    fifth_grads = torch.autograd.grad(fourth_grads, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=False)  


    fifth_grads_array = np.array(fifth_grads[0].data)

    causal_effect += ((1.0/24.0)*float(fifth_grads_array[feature_index])*mean_data[feature_index]*mean_data[feature_index]*mean_data[feature_index]*mean_data[feature_index]) + ((1.0/6.0)*float(fifth_grads_array[feature_index])*mean_data[feature_index]*mean_data[feature_index]) + ((1.0/6.0)*float(fifth_grads_array[feature_index])*mean_data[feature_index]*mean_data[feature_index]) 
    causal_effect_x += (0.5)*float(fifth_grads_array[feature_index])*mean_data[feature_index]*mean_data[feature_index]
    causal_effect_x_2 -= 0.5*float(fifth_grads_array[feature_index])*mean_data[feature_index]
    causal_effect_x_3 = (1.0/6.0)*float(fifth_grads_array[feature_index])
    causal_effect_x_4 = (1.0/24.0)*float(fifth_grads_array[feature_index])


    grad_mask = torch.zeros(first_grad_shape)
    grad_mask[feature_index] = 1.0

    sixth_grads = torch.autograd.grad(fifth_grads, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=False)  


    sixth_grads_array = np.array(sixth_grads[0].data)

    grad_mask = torch.zeros(first_grad_shape)
    grad_mask[feature_index] = 1.0

    seventh_grads = torch.autograd.grad(sixth_grads, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=True, only_inputs=True, allow_unused=False)  


    seventh_grads_array = np.array(seventh_grads[0].data)


    grad_mask = torch.zeros(first_grad_shape)
    grad_mask[feature_index] = 1.0

    eight_grads = torch.autograd.grad(seventh_grads, input_torchvar, grad_outputs=grad_mask, retain_graph=True, create_graph=False, only_inputs=True, allow_unused=False)  


    eight_grads_array = np.array(eight_grads[0].data)


    print causal_effect_x_4, "x^4 +", causal_effect_x_3, "x^3 +", causal_effect_x_2, "x^2 +", causal_effect_x, "x +", causal_effect
    plt.plot(np.linspace(start,end,100), np.array([(lambda x, a: float(first_grads[0][feature_index].data) + second_grads_array[feature_index]*(x - a) + ((1.0/2.0)*third_grads_array[feature_index]*(x - a)**2) + ((1.0/6.0)*fourth_grads_array[feature_index]*(x - a)**3) + ((1.0/24.0)*fifth_grads_array[feature_index]*(x - a)**4) + ((1.0/120.0)*sixth_grads_array[feature_index]*(x - a)**5) + ((1.0/720.0)*seventh_grads_array[feature_index]*(x - a)**6) + ((1.0/5040.0)*eight_grads_array[feature_index]*(x - a)**7))(val, mean_data[feature_index]) for val in np.linspace(start,end,100)]), label="feature " + str(feature_index))

    #plt.plot(np.linspace(start,end,100), np.array([(lambda x: causal_effect_x_2*(x**2) + causal_effect_x*x + causal_effect)(val) for val in np.linspace(start,end,100)]), label="class " + str(class_name[effect]))
    plt.legend()


for class_ind in range(3):
    effect = (class_ind)
    for feature_index in range(means.shape[1]):
        causal_factor = (feature_index)
        start = 0.0 #feature_start[feature_index][0]
        end = 1.0 #feature_start[feature_index][1]
        #print "Causal strength of", causal_factor, "for class", effect, ":",
        print effect, means[effect]
        causal_strength(model, feature_index=causal_factor, output_index = effect, mean_data = means[effect], cov_data = np.array(cov), start=start, end=end)
    plt.ylabel("causal strength on class label")
    plt.xlabel("value of cause variable")
    plt.title(class_name[effect])
    plt.savefig(str(class_name[effect]) + "_double_" + sys.argv[1] + ".png", dpi=300)
    #plt.savefig(str(feature_name[feature_index]) + ".png", dpi=300)
    #plt.show()
    plt.clf()    